In [11]:
pip install csv_reader

Note: you may need to restart the kernel to use updated packages.


In [99]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib 
matplotlib.rcParams['font.family'] = 'AppleGothic'
from IPython.display import Image 

In [100]:
data_1 = pd.read_csv('LPOINT_BIG_COMP_01_DEMO.csv')
data_2 = pd.read_csv('LPOINT_BIG_COMP_02_PDDE.csv')
data_3 = pd.read_csv('LPOINT_BIG_COMP_03_COP_U.csv')
data_4 = pd.read_csv('LPOINT_BIG_COMP_04_PD_CLAC.csv')
data_5 = pd.read_csv('LPOINT_BIG_COMP_05_BR.csv')
data_6 = pd.read_csv('LPOINT_BIG_COMP_06_LPAY.csv')

target_df = data_2
target_cust_df = data_1

/var/folders/mt/h66m2ngd2018sg3lkd5ghq200000gn/T/ipykernel_2050/1655262613.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2 = pd.read_csv('LPOINT_BIG_COMP_02_PDDE.csv')


## 거주지 대분류, 나이대, 성별 Feature 생성

In [101]:
data_1['ma_fem_dv'].loc[data_1['ma_fem_dv'] == "여성"] = 0
data_1['ma_fem_dv'].loc[data_1['ma_fem_dv'] == "남성"] = 1
data_1 = data_1.rename(columns = {'ma_fem_dv' : 'male'})
data_1['ages'] = data_1['ages'].str.replace(pat=r'[ㄱ-ㅣ가-힣]+', repl= r'', regex=True)
# data_1['zon_hlv'] = data_1['zon_hlv'].str.replace(pat=r'[a-zA-Z]+', repl= r'', regex=True)
data_1


,cust,male,ages,zon_hlv
0,M000034966,0,40,Z07
1,M000059535,0,30,Z12
2,M000136117,0,30,Z11
3,M000201112,0,50,Z17
4,M000225114,0,40,Z17
...,...,...,...,...
29908,M999708287,0,40,Z10
29909,M999770689,0,30,Z16
29910,M999849895,0,20,Z04
29911,M999926092,1,30,Z08


## 주요 방문지점 Feature 생성

In [102]:
visit_df = pd.merge(data_5, data_2, on = 'br_c', how = 'right')
visit_df = visit_df.groupby('cust')['zon_hlv'].agg(**{
    'most_common_visiting':lambda x:x.mode()
}).reset_index()
visit_df

data_1 = pd.merge(data_1 , visit_df, on = 'cust', how = 'left')
data_1 = data_1.dropna()
data_1

,cust,male,ages,zon_hlv,most_common_visiting
0,M000034966,0,40,Z07,Z07
2,M000136117,0,30,Z11,Z14
3,M000201112,0,50,Z17,Z10
4,M000225114,0,40,Z17,Z17
5,M000261625,0,40,Z17,Z17
...,...,...,...,...,...
29906,M999599111,0,40,Z04,Z04
29907,M999673157,1,30,Z10,Z10
29909,M999770689,0,30,Z16,Z16
29910,M999849895,0,20,Z04,Z10


## 주요 방문 매장 Feature 생성하기


In [103]:
store_df = data_2
store_df = store_df.groupby('cust')['cop_c'].agg(**{
    'most_common_store':lambda x:x.mode()
}).reset_index()
store_df

data_1 = pd.merge(data_1, store_df, on = 'cust', how = 'left')
data_1 = data_1.dropna()
data_1

,cust,male,ages,zon_hlv,most_common_visiting,most_common_store
0,M000034966,0,40,Z07,Z07,A02
1,M000136117,0,30,Z11,Z14,A01
2,M000201112,0,50,Z17,Z10,A04
3,M000225114,0,40,Z17,Z17,A01
4,M000261625,0,40,Z17,Z17,A01
...,...,...,...,...,...,...
26912,M999599111,0,40,Z04,Z04,A02
26913,M999673157,1,30,Z10,Z10,"[A01, A04]"
26914,M999770689,0,30,Z16,Z16,A04
26915,M999849895,0,20,Z04,Z10,A02


## 최대 구매액 Feature 생성하기

In [104]:
max_buy_df = data_2
max_buy_df = max_buy_df.sort_values(by = 'buy_am', ascending = False)
max_buy_df = max_buy_df.groupby('cust').head(1)
max_buy_df = max_buy_df[['cust', 'buy_am']]

data_1 = pd.merge(data_1 , max_buy_df, on = 'cust', how = 'left')
data_1 = data_1.dropna()
data_1 = data_1.rename(columns = {'buy_am' : 'max_buy_am'})
data_1

,cust,male,ages,zon_hlv,most_common_visiting,most_common_store,max_buy_am
0,M000034966,0,40,Z07,Z07,A02,65700.0
1,M000136117,0,30,Z11,Z14,A01,2612000.0
2,M000201112,0,50,Z17,Z10,A04,8000.0
3,M000225114,0,40,Z17,Z17,A01,129000.0
4,M000261625,0,40,Z17,Z17,A01,1160000.0
...,...,...,...,...,...,...,...
26912,M999599111,0,40,Z04,Z04,A02,1158000.0
26913,M999673157,1,30,Z10,Z10,"[A01, A04]",1673000.0
26914,M999770689,0,30,Z16,Z16,A04,59000.0
26915,M999849895,0,20,Z04,Z10,A02,269000.0


## 주요 구매 시간대

In [105]:
data_2

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1
3,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1
4,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1
...,...,...,...,...,...,...,...,...,...,...
4381738,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1
4381739,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1
4381740,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1
4381741,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1


In [106]:
time_df = data_2
time_df = time_df.groupby('cust')['de_hr'].agg(**{
    'most_common_time':lambda x:x.mode()
}).reset_index()

data_1 = pd.merge(data_1 , time_df, on = 'cust', how = 'left')
data_1 = data_1.dropna()
data_1

,cust,male,ages,zon_hlv,most_common_visiting,most_common_store,max_buy_am,most_common_time
0,M000034966,0,40,Z07,Z07,A02,65700.0,16
1,M000136117,0,30,Z11,Z14,A01,2612000.0,15
2,M000201112,0,50,Z17,Z10,A04,8000.0,15
3,M000225114,0,40,Z17,Z17,A01,129000.0,18
4,M000261625,0,40,Z17,Z17,A01,1160000.0,18
...,...,...,...,...,...,...,...,...
26912,M999599111,0,40,Z04,Z04,A02,1158000.0,19
26913,M999673157,1,30,Z10,Z10,"[A01, A04]",1673000.0,"[1, 12, 19]"
26914,M999770689,0,30,Z16,Z16,A04,59000.0,21
26915,M999849895,0,20,Z04,Z10,A02,269000.0,19


## L.Pay 이용 유무

In [107]:
data_6 = pd.read_csv('LPOINT_BIG_COMP_06_LPAY.csv')
lpay_df = data_6.groupby('cust').count()
lpay_df = lpay_df.reset_index()
lpay_df['Lpay_user'] = int(1)
lpay_df = lpay_df[['cust', 'Lpay_user']]
lpay_df

,cust,Lpay_user
0,M000136117,1
1,M000419293,1
2,M000494848,1
3,M000557840,1
4,M000871427,1
...,...,...
8901,M999227380,1
8902,M999268192,1
8903,M999599111,1
8904,M999770689,1


In [108]:
data_1 = pd.merge(data_1 , lpay_df, on = 'cust', how = 'left')
data_1['Lpay_user'] = data_1['Lpay_user'].fillna(0).astype(int)
data_1

,cust,male,ages,zon_hlv,most_common_visiting,most_common_store,max_buy_am,most_common_time,Lpay_user
0,M000034966,0,40,Z07,Z07,A02,65700.0,16,0
1,M000136117,0,30,Z11,Z14,A01,2612000.0,15,1
2,M000201112,0,50,Z17,Z10,A04,8000.0,15,0
3,M000225114,0,40,Z17,Z17,A01,129000.0,18,0
4,M000261625,0,40,Z17,Z17,A01,1160000.0,18,0
...,...,...,...,...,...,...,...,...,...
26912,M999599111,0,40,Z04,Z04,A02,1158000.0,19,1
26913,M999673157,1,30,Z10,Z10,"[A01, A04]",1673000.0,"[1, 12, 19]",0
26914,M999770689,0,30,Z16,Z16,A04,59000.0,21,1
26915,M999849895,0,20,Z04,Z10,A02,269000.0,19,0


## 2번 이상 구매한 상품 종류 수 


In [109]:
rebuy_df = data_2
rebuy_df = rebuy_df.groupby(['cust', 'pd_c']).count()
rebuy_df = rebuy_df.rct_no >= 2
rebuy_df = rebuy_df.reset_index()
rebuy_df

,cust,pd_c,rct_no
0,M000034966,PD0116,False
1,M000034966,PD0169,False
2,M000034966,PD0178,False
3,M000034966,PD0204,False
4,M000034966,PD0218,True
...,...,...,...
1637596,M999962961,PD1857,False
1637597,M999962961,PD1889,True
1637598,M999962961,PD1896,False
1637599,M999962961,PD1918,False


In [110]:
rebuy_df = rebuy_df.groupby('cust').sum()
rebuy_df = rebuy_df.rename(columns = {'rct_no' : 'twice_product'})
rebuy_df = rebuy_df.reset_index()
rebuy_df

,cust,twice_product
0,M000034966,10
1,M000136117,15
2,M000201112,2
3,M000225114,28
4,M000261625,16
...,...,...
26912,M999599111,6
26913,M999673157,5
26914,M999770689,50
26915,M999849895,10


In [111]:
data_1 = pd.merge(data_1 , rebuy_df, on = 'cust', how = 'left')
data_1

,cust,male,ages,zon_hlv,most_common_visiting,most_common_store,max_buy_am,most_common_time,Lpay_user,twice_product
0,M000034966,0,40,Z07,Z07,A02,65700.0,16,0,10
1,M000136117,0,30,Z11,Z14,A01,2612000.0,15,1,15
2,M000201112,0,50,Z17,Z10,A04,8000.0,15,0,2
3,M000225114,0,40,Z17,Z17,A01,129000.0,18,0,28
4,M000261625,0,40,Z17,Z17,A01,1160000.0,18,0,16
...,...,...,...,...,...,...,...,...,...,...
26912,M999599111,0,40,Z04,Z04,A02,1158000.0,19,1,6
26913,M999673157,1,30,Z10,Z10,"[A01, A04]",1673000.0,"[1, 12, 19]",0,5
26914,M999770689,0,30,Z16,Z16,A04,59000.0,21,1,50
26915,M999849895,0,20,Z04,Z10,A02,269000.0,19,0,10


In [112]:
data_1.to_csv('customer_feature.csv')

## RFM 수치 계산 (Target값 구하기)

In [113]:
target_df_buy = target_df.groupby('cust').sum()
target_df_buy = target_df_buy.reset_index()
target_df_buy

,cust,chnl_dv,de_dt,de_hr,buy_am,buy_ct
0,M000034966,51,1030749070,854,654120.0,63
1,M000136117,122,2202951361,1673,27373630.0,114
2,M000201112,21,424427205,249,53120.0,30
3,M000225114,159,3213500596,2779,1937700.0,182
4,M000261625,67,1354116249,1039,6052700.0,71
...,...,...,...,...,...,...
26912,M999599111,36,687158269,617,1484022.0,43
26913,M999673157,24,485057899,332,2851370.0,28
26914,M999770689,332,6366349111,4196,1187140.0,385
26915,M999849895,72,1455173167,1148,1103792.0,119


In [114]:
target_cust_df

,cust,ma_fem_dv,ages,zon_hlv
0,M000034966,0,40대,Z07
1,M000059535,0,30대,Z12
2,M000136117,0,30대,Z11
3,M000201112,0,50대,Z17
4,M000225114,0,40대,Z17
...,...,...,...,...
29908,M999708287,0,40대,Z10
29909,M999770689,0,30대,Z16
29910,M999849895,0,20대,Z04
29911,M999926092,1,30대,Z08


In [115]:
target_cust_df = target_cust_df.drop(columns = ['ma_fem_dv', 'ages', 'zon_hlv'])
target_cust_df

,cust
0,M000034966
1,M000059535
2,M000136117
3,M000201112
4,M000225114
...,...
29908,M999708287
29909,M999770689
29910,M999849895
29911,M999926092


### Monetary 구하기

In [116]:
target_cust_df = pd.merge(target_cust_df, target_df_buy[['cust', 'buy_am']], on = 'cust', how = 'outer')
target_cust_df = target_cust_df.fillna(0)
target_cust_df = target_cust_df.rename(columns = {'buy_am' : 'monetary'})
target_cust_df

,cust,monetary
0,M000034966,654120.0
1,M000059535,0.0
2,M000136117,27373630.0
3,M000201112,53120.0
4,M000225114,1937700.0
...,...,...
29908,M999708287,0.0
29909,M999770689,1187140.0
29910,M999849895,1103792.0
29911,M999926092,0.0


### Recency 구하기

In [117]:
target_df_rec = target_df
target_df_rec = target_df_rec.sort_values(by = 'de_dt', ascending = True)
target_df_rec = target_df_rec.groupby('cust').tail(1)
target_df_rec

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct
705614,M352925752,A02246000515,1,A02,A020074,PD0393,20210101,13,1850.0,1
4297417,M583636385,A05000018631,1,A05,A050382,PD0544,20210101,14,109000.0,1
4305836,M500652077,E06000279741,2,A06,NaN,PD0073,20210101,23,84000.0,1
4305690,M133778155,E06000174291,2,A06,NaN,PD0045,20210101,15,2500.0,1
4305702,M586981937,E06000184359,2,A06,NaN,PD1857,20210101,15,21900.0,1
...,...,...,...,...,...,...,...,...,...,...
3069216,M618622383,A02128050650,1,A02,A020030,PD0350,20211231,21,3990.0,1
3069243,M579847593,A02128053258,1,A02,A020030,PD1387,20211231,19,4580.0,1
3069229,M815601317,A02128051047,1,A02,A020030,PD0401,20211231,12,5340.0,3
3069236,M939657690,A02128052231,1,A02,A020030,PD1179,20211231,19,1890.0,1


In [118]:
from datetime import datetime

In [119]:
# int형으로 된 de_dt를 문자형으로 바꾸기
target_df_rec['de_dt'] = target_df_rec['de_dt'].astype(str)
target_df_rec['de_dt'] = target_df_rec['de_dt'].str[0:4] + "-" + target_df_rec['de_dt'].str[4:6] + "-" + target_df_rec['de_dt'].str[6:8]
target_df_rec['de_dt'] = target_df_rec['de_dt'].astype('datetime64[ns]')

target_df_rec['recency'] = datetime.strptime("20220101", "%Y%m%d") - target_df_rec['de_dt']
target_df_rec = target_df_rec[['cust', 'recency']]
target_df_rec

,cust,recency
705614,M352925752,365 days
4297417,M583636385,365 days
4305836,M500652077,365 days
4305690,M133778155,365 days
4305702,M586981937,365 days
...,...,...
3069216,M618622383,1 days
3069243,M579847593,1 days
3069229,M815601317,1 days
3069236,M939657690,1 days


In [120]:
target_cust_df = pd.merge(target_cust_df, target_df_rec, on = 'cust', how = 'left')
target_cust_df

,cust,monetary,recency
0,M000034966,654120.0,9 days
1,M000059535,0.0,NaT
2,M000136117,27373630.0,2 days
3,M000201112,53120.0,34 days
4,M000225114,1937700.0,1 days
...,...,...,...
29908,M999708287,0.0,NaT
29909,M999770689,1187140.0,1 days
29910,M999849895,1103792.0,31 days
29911,M999926092,0.0,NaT


### Frequency 구하기

In [121]:
target_df_fre = target_df

# 같은 영수증 넘버를 가진 다른 상품 -> 구매 횟수 1번으로 인정, 모두 중복 제거
target_df_fre = target_df_fre.drop_duplicates(subset = ['rct_no'])
target_df_fre = target_df_fre.groupby('cust').count()
target_df_fre = target_df_fre.reset_index()

# 영수증의 개수 -> 구매 횟수로 인정
target_df_fre = target_df_fre[['cust', 'rct_no']]
target_df_fre = target_df_fre.rename(columns = {'rct_no' : 'frequency'})
target_df_fre

,cust,frequency
0,M000034966,12
1,M000136117,70
2,M000201112,5
3,M000225114,78
4,M000261625,48
...,...,...
26912,M999599111,8
26913,M999673157,17
26914,M999770689,122
26915,M999849895,34


In [122]:
target_cust_df = pd.merge(target_cust_df, target_df_fre, on = 'cust', how = 'left')
target_cust_df

,cust,monetary,recency,frequency
0,M000034966,654120.0,9 days,12.0
1,M000059535,0.0,NaT,NaN
2,M000136117,27373630.0,2 days,70.0
3,M000201112,53120.0,34 days,5.0
4,M000225114,1937700.0,1 days,78.0
...,...,...,...,...
29908,M999708287,0.0,NaT,NaN
29909,M999770689,1187140.0,1 days,122.0
29910,M999849895,1103792.0,31 days,34.0
29911,M999926092,0.0,NaT,NaN


In [123]:
target_cust_df['recency'] = target_cust_df['recency'].dt.days
target_cust_df

,cust,monetary,recency,frequency
0,M000034966,654120.0,9.0,12.0
1,M000059535,0.0,NaN,NaN
2,M000136117,27373630.0,2.0,70.0
3,M000201112,53120.0,34.0,5.0
4,M000225114,1937700.0,1.0,78.0
...,...,...,...,...
29908,M999708287,0.0,NaN,NaN
29909,M999770689,1187140.0,1.0,122.0
29910,M999849895,1103792.0,31.0,34.0
29911,M999926092,0.0,NaN,NaN


# 결측치 처리

분석에 영향을 주는 결측치를 처리하기 위해 Monetary, Frequency는 가장 낮은 점수값인 0을, Recency는 가장 낮은 점수값인 366 days를 입력한다.

In [124]:
target_cust_df[['monetary', 'frequency']] = target_cust_df[['monetary', 'frequency']].fillna(0)
target_cust_df['recency'] = target_cust_df['recency'].fillna(366)

In [125]:
target_cust_df.sort_values(by = 'recency', ascending = False)

,cust,monetary,recency,frequency
4119,M138108319,0.0,366.0,0.0
25562,M851430554,0.0,366.0,0.0
14423,M480688070,0.0,366.0,0.0
15686,M523011190,0.0,366.0,0.0
19437,M646272710,0.0,366.0,0.0
...,...,...,...,...
25337,M843199447,28202720.0,1.0,92.0
10020,M333112427,151600.0,1.0,9.0
9997,M332539888,5941418.0,1.0,42.0
9994,M332499386,601320.0,1.0,23.0


In [126]:
target_cust_df.to_csv('Target.csv')

In [127]:
monetary = target_cust_df['monetary']

for x in np.linspace(0,1,6)[1:-1]:
    print(f'분위수 : {np.quantile(monetary,x):.1f}')

분위수 : 139182.0
분위수 : 742764.8
분위수 : 1688676.8
분위수 : 3712017.6


In [128]:
recency = target_cust_df['recency']

for x in np.linspace(0,1,6)[1:-1]:
    print(f'분위수 : {np.quantile(recency,x):.1f}')

분위수 : 4.0
분위수 : 11.0
분위수 : 33.0
분위수 : 147.0


In [129]:
frequency = target_cust_df['frequency']

for x in np.linspace(0,1,6)[1:-1]:
    print(f'분위수 : {np.quantile(frequency,x):.1f}')

분위수 : 3.0
분위수 : 20.0
분위수 : 35.0
분위수 : 68.0
